In [1]:
!gsutil cp gs://dpo_cnil/me_parameters/dpo_cnil_me.json ../vector_store_me_parameters/

Copying gs://dpo_cnil/me_parameters/dpo_cnil_me.json...
/ [1 files][  385.0 B/  385.0 B]                                                
Operation completed over 1 objects/385.0 B.                                      


In [1]:
import sys

# Add the directory to sys.path
sys.path.append('/home/qianyucazelles/internal_knowledge_chatbot/src')
from utils.doc_to_vertex_search import *
from utils.general_utils import *

from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
import os

In [4]:
import nltk
def nb_tokens(text):
    return len(nltk.word_tokenize(text))

In [3]:
def load_pdf_to_langchain_doc(
    pdf_path,
    unstructure_loader: bool=False):
    
    if unstructure_loader:
        print("Using unstrucure loader")
        loader = UnstructuredPDFLoader(f'{pdf_path}')
    else:
        loader = PyPDFLoader(f'{pdf_path}')
    doc = loader.load()
    
    return doc

In [7]:
!ls ../documents/ecg_dpo/CNIL/

'CHAPITRE I - Dispositions générales.pdf'
'CHAPITRE II - Principes _ CNIL.pdf'
'CHAPITRE III - Droits de la personne concernée _ CNIL.pdf'
'CHAPITRE IV - Responsable du traitement et sous-traitant _ CNIL.pdf'
'CHAPITRE IX - Dispositions relatives à des situations particulières de traitement _ CNIL.pdf'
'CHAPITRE V - Transferts de données à caractère personnel vers des pays tiers ou à des organisations internationales _ CNIL.pdf'
'CHAPITRE VI - Autorités de contrôle indépendantes _ CNIL.pdf'
'CHAPITRE VII - Coopération et cohérence _ CNIL.pdf'
'CHAPITRE VIII - Voies de recours, responsabilité et sanctions _ CNIL.pdf'
"CHAPITRE X - Actes délégués et actes d'exécution _ CNIL.pdf"
'CHAPITRE XI - Dispositions finales _ CNIL.pdf'


In [4]:
doc = load_pdf_to_langchain_doc('../documents/ecg_dpo/CNIL/CHAPITRE XI - Dispositions finales _ CNIL.pdf')

In [7]:
len(doc)

3

In [5]:
doc_to_chunk(doc, recusive=False)

Created a chunk of size 1569, which is longer than the specified 1500


[Document(page_content="4/15/24, 9:58 AM CHAPITRE XI - Dispositions finales | CNIL\nhttps://www.cnil.fr/fr/reglement-europeen-protection-donnees/chapitre11 1/31.\n2.CHAPITRE XI - Dispositions ﬁnales\nArticle 94 - Abrogation de la directive 95/46/CE\nLa directive 95/46/CE est abrogée avec effet au 25 mai 2018.\nLes références faites à la directive abrogée s'entendent comme faites au présent règlement. Les références\nfaites au groupe de protection des personnes à l'égard du traitement des données à caractère personnel\ninstitué par l'article 29 de la directive 95/46/CE s'entendent comme faites au comité européen de la\nprotection des données institué par le présent règlement.\nArticle 95 - Relation avec la directive 2002/58/CEArticle 94 - Abrogation de la directive 95/46/CE \nArticle 95 - Relation avec la directive 2002/58/CE \nArticle 96 - Relation avec les accords conclus antérieurement \nArticle 97 - Rapports de la Commission. \nArticle 98 - Réexamen d'autres actes juridiques de l'Un

In [8]:
doc_to_chunk(doc, recusive=True, chunk_size= 500,chunk_overlap=20)

[Document(page_content="4/15/24, 9:58 AM CHAPITRE XI - Dispositions finales | CNIL\nhttps://www.cnil.fr/fr/reglement-europeen-protection-donnees/chapitre11 1/31.\n2.CHAPITRE XI - Dispositions ﬁnales\nArticle 94 - Abrogation de la directive 95/46/CE\nLa directive 95/46/CE est abrogée avec effet au 25 mai 2018.\nLes références faites à la directive abrogée s'entendent comme faites au présent règlement", metadata={'source': '../documents/ecg_dpo/CNIL/CHAPITRE XI - Dispositions finales _ CNIL.pdf', 'page': 0, 'chunk': 0}),
 Document(page_content=". Les références\nfaites au groupe de protection des personnes à l'égard du traitement des données à caractère personnel\ninstitué par l'article 29 de la directive 95/46/CE s'entendent comme faites au comité européen de la\nprotection des données institué par le présent règlement.\nArticle 95 - Relation avec la directive 2002/58/CEArticle 94 - Abrogation de la directive 95/46/CE", metadata={'source': '../documents/ecg_dpo/CNIL/CHAPITRE XI - Dispos

In [19]:
def pdf_to_vector_search(
    pdf_path,
    me,
    unstructure_loader: bool=False,
    split_to_article: bool=False
):
    print_with_time(f"Process file:{pdf_path} ")
    
    #load pages
    pages = load_pdf_to_langchain_doc(pdf_path,unstructure_loader)
    
    print_with_line(f"len of pages:{len(pages)}")
    
    #split to article
    if split_to_article:
        articles = split_pages_into_artiles(pages)
        doc_splits = doc_to_chunk(articles, recusive=True, chunk_size= 500,chunk_overlap=20)
    else:
        doc_splits = doc_to_chunk(pages, recusive=True, chunk_size= 500,chunk_overlap=20)
        

    print_with_line(f"n splits:{len(doc_splits)}")
    
    #add to vector store
    doc_ids = add_splits_to_vector_store(doc_splits,me)
    
    
    #log process
    if len(doc_ids)>0:
        
        print_with_line(f"Added to vector store with {len(doc_ids)} ids.")
        
        with open("../add_docs_to_vectorstore_logs/processed_pdfs.txt", "a") as f:
            f.write(f"'{pdf_path}',\n")
        with open("../add_docs_to_vectorstore_logs/processed_pdfs_doc_ids.txt", "a") as f:
            f.write(f"'{pdf_path}': {doc_ids},\n")
        
    else:
        
        print_with_star("No doc ids returned")
        
        with open(f"../add_docs_to_vectorstore_logs/unprocessed_pdfs.txt", "a") as f:
            f.write(f"'{pdf_path}'," + "\n")
            
    return doc_ids

In [16]:
def process_folder(me,CHATBOT_ID, folder_path, pdf_doc_ids, other_files):
    try:
        processed_pdfs = read_txt_to_list("../add_docs_to_vectorstore_logs/processed_pdfs.txt") 
    except:
        processed_pdfs = []    
    for item in os.listdir(folder_path):
        
        item_path = os.path.join(folder_path, item)
        print_with_time(f"Scanning {item_path}")
        
        if os.path.isdir(item_path):
            
            print_with_time(f"{item_path} is folder, now go to new level")
            process_folder(me,CHATBOT_ID,item_path,pdf_doc_ids, other_files)
            
        elif item.lower().endswith('.pdf'):
            if item not in processed_pdfs:
                
                print_with_time(f"{item_path} is pdf, now process to vector store")
                try:
                    
                    doc_ids = pdf_to_vector_search(item_path,me)
                    
                except Exception as e:
                    with open(f"../add_docs_to_vectorstore_logs/unprocessed_pdfs.txt", "a") as f:
                        f.write(f"'{item_path}'," + "\n")
                    print_with_star(f"An error occurred while processing {item_path}: {e}")
                    doc_ids=[]
                pdf_doc_ids[item_path]=doc_ids
            
        else:
            other_files.append(item_path)

In [17]:
def main(me,
         CHATBOT_ID: str,
         root_folder: str
         ):
    
    pdf_data = {}
    other_files = []
    
    process_folder(me,CHATBOT_ID,root_folder, pdf_data, other_files)
    
    # Writing PDF data to JSON file
    pdf_output_file = f"../add_docs_to_vectorstore_logs/{CHATBOT_ID}_pdf_doc_ids.json"
    with open(pdf_output_file, "w") as json_file:
        json.dump(pdf_data, json_file, indent=4)
    print(f"PDF data has been written to {pdf_output_file}")
    
    # Writing other files list to JSON file
    other_output_file = f"../add_docs_to_vectorstore_logs/{CHATBOT_ID}_non_pdf_files.json"
    with open(other_output_file, "w") as json_file:
        json.dump(other_files, json_file, indent=4)
    print(f"Non-PDF files list has been written to {other_output_file}")



In [13]:
root_folder = f"../documents/ecg_dpo/CNIL"

In [20]:
!rm ../add_docs_to_vectorstore_logs/processed_pdfs.txt

rm: cannot remove '../add_docs_to_vectorstore_logs/processed_pdfs.txt': No such file or directory


In [21]:
!rm ../add_docs_to_vectorstore_logs/unprocessed_pdfs.txt

In [22]:
if __name__ == "__main__":
    CHATBOT_ID = "ecg_cnil"
    # root_folder = f"../documents/{CHATBOT_ID}/"
    # get embeddings
    embeddings = get_embeddings()

    #get parameters
    parameters = get_me_parameters(f'../vector_store_me_parameters/{CHATBOT_ID}_me.json')

    #get vector store
    me = get_vector_store(parameters, embeddings)
    
    main(me,CHATBOT_ID,root_folder)

Model_name will become a required arg for VertexAIEmbeddings starting from Feb-01-2024. Currently the default is set to textembedding-gecko@001


[2024-05-14 12:39:03] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE II - Principes _ CNIL.pdf
[2024-05-14 12:39:03] ../documents/ecg_dpo/CNIL/CHAPITRE II - Principes _ CNIL.pdf is pdf, now process to vector store
[2024-05-14 12:39:03] Process file:../documents/ecg_dpo/CNIL/CHAPITRE II - Principes _ CNIL.pdf 
----------------------------------------------------------------------------------------------------
[2024-05-14 12:39:03] len of pages:7
----------------------------------------------------------------------------------------------------
[2024-05-14 12:39:03] n splits:33
Waiting
......successfully added to vector store with 33 new doc ids
----------------------------------------------------------------------------------------------------
[2024-05-14 12:39:11] Added to vector store with 33 ids.
[2024-05-14 12:39:11] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE I - Dispositions générales.pdf
[2024-05-14 12:39:11] ../documents/ecg_dpo/CNIL/CHAPITRE I - Dispositions générales.pdf is pdf,

In [1]:
"textembedding-gecko-multilingual@001"

'textembedding-gecko-multilingual@001'

In [2]:
from langchain.embeddings import VertexAIEmbeddings

/home/qianyucazelles/internal_knowledge_chatbot/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.vertexai.VertexAIEmbeddings` was deprecated in langchain-community 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAIEmbeddings`.
  warn_deprecated(
2024-05-14 13:27:06.111763: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


VertexAIEmbeddings(project=None, location='us-central1', request_parallelism=5, max_retries=6, stop=None, model_name='textembedding-gecko-multilingual@001', client=<vertexai.language_models.TextEmbeddingModel object at 0x7fa37b426950>, client_preview=None, temperature=0.0, max_output_tokens=128, top_p=0.95, top_k=40, credentials=None, n=1, streaming=False, instance={'max_batch_size': 250, 'batch_size': 250, 'min_batch_size': 5, 'min_good_batch_size': 5, 'lock': <unlocked _thread.lock object at 0x7fa3a53cc540>, 'batch_size_validated': False, 'task_executor': <concurrent.futures.thread.ThreadPoolExecutor object at 0x7fa3799e6140>, 'embeddings_task_type_supported': True}, show_progress_bar=False)